In [ ]:
import pandas as pd
import json

from serlo.mysql import db

In [ ]:
class entity:
    def __init__(self, id, empty_boxes):
        self.id = id
        self.empty_boxes = empty_boxes
        
    

In [ ]:
#def get_items(obj):

def get_keys(obj, stack):
    #print("this: ")
    #print(obj)
    for k, v in obj.items():
      k2 = ([k] if k else []) + stack # don't return empty keys
      if v and isinstance(v, dict):
        for c in get_keys(v, k2):
          yield c
      else: # leaf
        yield k2

def get_values(obj):
    for v in obj.values():
      if not v: continue
      if isinstance(v, dict):
        for c in get_values(v):
          yield c
      else: # leaf
        yield v if isinstance(v, list) else [v]
        
def get_value_from_key_array(obj, keys):
    #print(keys)
    if len(keys) == 0:
        return
    if len(keys) == 1:
        return obj[keys[0]]
    else:
        key = keys.pop(-1)
        #print(key)
        return get_value_from_key_array(obj[key], keys)

  #return list(get_keys(obj,[])), list(get_values(obj))

In [ ]:
def find_empty_boxes(obj):
    empty_boxes = []
    #print(obj)
    keys = list(get_keys(obj, []))
    #print("keys")
    #print(keys)
    for i in range(len(keys)):
        leaf = keys[i][0]
        if leaf == "plugin":
            plugin = {}
            if len(keys[i]) == 1:
                plugin = obj
            else:
                #print("test")
                plugin = get_value_from_key_array(obj, keys[i][1:])
            if plugin["plugin"] == "box":
                #print("box")
                #print(plugin)
                if is_empty_box(plugin["state"]):
                    empty_boxes.append(plugin["state"]["anchorId"])
                    #print(1)
        elif leaf == "state":
            #print(i)
            #print(keys[i])
            #print("value")
            #print(get_value_from_key_array(obj, keys[i]))
            state = get_value_from_key_array(obj, keys[i])
            if type(state) == list:
                for sub_obj in state:
                    print("subobj: ")
                    print(sub_obj)
                    #print("theses: ")
                    #print(find_empty_boxes(sub_obj))
                    empty_boxes.extend(find_empty_boxes(sub_obj))
    return empty_boxes
                    
def handle_boxes(boxes):
    if boxes:
        for box in boxes:
            if is_empty_box(box):
                handle_empty_box(box)
                    
                    
def is_empty_box(box):
    keys = list(get_keys(box, []))
    for i in range(len(keys)):
        leaf = keys[i][0]
        if leaf == "text":
            if get_value_from_key_array(box, keys[i]):
                return False
    return True
    
    
    
#def is_empty_box(box):
#    return is_empty_object(box["content"])

def is_empty_object(obj):
    #print("plugin")
    #print(obj)
    match obj["plugin"]:
        case "rows":
            for row in obj["state"]:
                if not is_empty_object(row):
                    return False
        case "text":
            for text in obj["state"]:
                if text["type"] == 'p':
                    return is_empty_text(text["children"])
                else:
                    print("unhandled text type: ")
                    print(text["type"])
                    return False
        case _:
            print("unhandled plugin: ")
            print(obj["plugin"])
            return False
            
    return True
    
def is_empty_text(text):
    #print("children")
    #print(text)
    for child in text:
        if child["text"]:
            return True
    return False

In [ ]:
df = pd.read_sql("""
SELECT entity.id, value
FROM entity_revision_field, entity
where field = "content"
    AND entity.current_revision_id = entity_revision_field.entity_revision_id;
""", db)

df.head()

In [ ]:
entities = []
for i, revision in df.iterrows():
    #print("id")
    #print(revision["id"])
    try:
        currentObject = json.loads(revision["value"])
    except:
        print("id")
        continue
    if isinstance(currentObject, dict):
        empty_boxes = find_empty_boxes(currentObject)
        if len(empty_boxes):
            entities.append(entity(revision["id"], empty_boxes))
            

In [ ]:
entities[0].id

In [ ]:
#df["id" == 16078]
df[df.id == 1511]#["value"]
df["value"][8]

In [ ]:
entities[0].empty_boxes

In [ ]:
bytes(df["value"][400], "latin1").decode("utf8")

In [ ]:
test = '{"plugin":"article","state":{"introduction":{"plugin":"articleIntroduction","state":{"explanation":{"plugin":"text","state":[{"type":"p","children":[{"text":"In diesem Artikel werden Boxen und ihre Verwandten ausprobiert :)"}]}]},"multimedia":{"plugin":"image","state":{"src":"","caption":{"plugin":"text","state":[{"type":"p","children":[{}]}]}}},"illustrating":true,"width":50}},"content":{"plugin":"rows","state":[{"plugin":"text","state":[{"type":"p","children":[{"text":"Text Anfang"}]}]},{"plugin":"box","state":{"title":{"plugin":"text","state":[{"type":"p","children":[{}]}]},"content":{"plugin":"rows","state":[{"plugin":"text","state":[{"type":"p","children":[{"text":"Das hier ist ein Merksatz. In der Plugin-Beschreibung steht auch, sodass dieser bisher wohl nicht besonders semantisch verwendet wurde."}]},{"type":"p","children":[{"text":""}]}]}]},"type":"blank","anchorId":"box80970"}},{"plugin":"text","state":[{"type":"p","children":[{"text":"Zwischentext zwischen den Elementen"}]}]},{"plugin":"box","state":{"title":{"plugin":"text","state":[{"type":"p","children":[{}]}]},"content":{"plugin":"rows","state":[{"plugin":"text","state":[{"type":"p","children":[{"text":"Das hier ist ein Zitat. Angeblich ist es einfach eingerückt, oder passiert hier noch mehr?"}]},{"type":"p","children":[{"text":""}]}]}]},"type":"quote","anchorId":"box35521"}},{"plugin":"text","state":[{"type":"p","children":[{"text":"Noch ein schicker Zwischentext"}]}]},{"plugin":"spoiler","state":{"title":"Das ist der Spoiler-Titel","content":{"plugin":"rows","state":[{"plugin":"text","state":[{"type":"p","children":[{"text":"Das ist der Spoiler-Inhalt, hoffentlich."}]}]}]}}},{"plugin":"text","state":[{"type":"p","children":[{"text":"Und zum Abschluss kommt noch mal normaler Text"}]}]},{"plugin":"text","state":[{"type":"p","children":[{"text":"Und hier ein Experiment mit Tastendrücken: "},{"type":"math","src":"\\\\fbox {~1~} ~\\\\fbox{Enter}~\\\\fbox{~2~} ~\\\\fbox{Enter}~\\\\fbox{~+~}","inline":true,"children":[{"text":"1 Enter"}]},{"text":""}]}]},{"plugin":"text","state":[{"type":"p","children":[{}]}]},{"plugin":"box","state":{"type":"note","title":{"plugin":"text","state":[{"type":"p","children":[{}]}]},"anchorId":"box32917","content":{"plugin":"rows","state":[{"plugin":"text","state":[{"type":"p","children":[{"text":"Das ist eine schicke neue Box."}]}]}]}}}]},"exercises":[],"exerciseFolder":{"id":"","title":""},"relatedContent":{"articles":[],"courses":[],"videos":[]},"sources":[]}}'

In [ ]:
obj = json.loads(test)
keys = list(get_keys(obj, []))
keys

In [ ]:
key_arr = ['plugin', 'introduction', 'state']
value = get_value_from_key_array(obj, key_arr)
value